In [0]:
%sql
DROP SCHEMA IF EXISTS cpg_industry.gold_sales CASCADE;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS cpg_industry.gold_sales;

In [0]:
%sql
WITH OrderMetrics AS (
    SELECT
        COALESCE(SUM(total_amount), 0) AS total_consumer_sales,
        COALESCE(AVG(total_amount), 0) AS average_order_value,
        COALESCE(COUNT(order_id), 0) AS total_order_count
    FROM cpg_industry.silver.cpg_consumer_order
),
OrderFrequency AS (
    SELECT COALESCE(AVG(order_count), 0) AS order_frequency
    FROM (
        SELECT consumer_id, COUNT(order_id) AS order_count
        FROM cpg_industry.silver.cpg_consumer_order
        GROUP BY consumer_id
    ) sub
),
ActiveConsumers AS (
    SELECT COALESCE(COUNT(DISTINCT consumer_id), 0) AS active_consumer_count
    FROM cpg_industry.silver.cpg_consumer
    WHERE is_active = TRUE
)
SELECT
    'Consumer KPIs' AS kpi_name,
    ROUND(om.total_consumer_sales, 2) AS total_consumer_sales,
    ROUND(om.average_order_value, 2) AS average_order_value,
    ROUND(of.order_frequency, 2) AS order_frequency_per_consumer,
    ROUND(ac.active_consumer_count, 2) AS active_consumer_count,
    ROUND(om.total_order_count, 2) AS total_consumer_order_count
FROM OrderMetrics om
CROSS JOIN OrderFrequency of
CROSS JOIN ActiveConsumers ac;

In [0]:
%sql
WITH PurchaseMetrics AS (
    SELECT
        COALESCE(SUM(total_amount), 0) AS total_distributor_sales,
        COALESCE(AVG(total_amount), 0) AS average_purchase_value,
        COALESCE(COUNT(purchase_id), 0) AS total_purchase_count
    FROM cpg_industry.silver.cpg_distributor_purchases
),
ActiveDistributors AS (
    SELECT COALESCE(COUNT(DISTINCT distributor_id), 0) AS active_distributor_count
    FROM cpg_industry.silver.cpg_distributor
),
InventoryForTopProducts AS (
    SELECT COALESCE(SUM(i.quantity_on_hand), 0) AS total_inventory_top_products
    FROM cpg_industry.silver.cpg_inventory i
    WHERE i.product_id IN (
        SELECT product_id
        FROM cpg_industry.silver.cpg_distributor_purchase_items
        GROUP BY product_id
        ORDER BY SUM(total_price) DESC
        LIMIT 10
    )
)
SELECT
    'Distributor KPIs' AS kpi_name,
    ROUND(pm.total_distributor_sales, 2) AS total_distributor_sales,
    ROUND(pm.average_purchase_value, 2) AS average_purchase_value,
    ROUND(ad.active_distributor_count, 2) AS active_distributor_count,
    ROUND(pm.total_purchase_count, 2) AS total_distributor_purchase_count,
    ROUND(itp.total_inventory_top_products, 2) AS total_inventory_top_products
FROM PurchaseMetrics pm
CROSS JOIN ActiveDistributors ad
CROSS JOIN InventoryForTopProducts itp;

### Out_of_Stock

In [0]:
%sql
-- CREATE OR REPLACE  TABLE cpg_industry.gold_sales.out_of_stock AS
SELECT
    -- KPI calculation
    COUNT(DISTINCT CASE 
        WHEN quantity_on_hand = 0 OR inventory_status = 'Low Stock' 
        THEN product_id 
    END) AS stockout_count,
    
    COUNT(DISTINCT product_id) AS total_skus,
    
    ROUND(
        (COUNT(DISTINCT CASE 
            WHEN quantity_on_hand = 0 OR inventory_status = 'Low Stock' 
            THEN product_id 
        END) * 100.0) / COUNT(DISTINCT product_id),
        2
    ) AS out_of_stock_rate_percentage
FROM cpg_industry.silver.cpg_inventory
WHERE location_is_active = true;


In [0]:
%sql
-- CREATE OR REPLACE TABLE cpg_industry.gold_sales.gold_clv_consumer AS
SELECT
    c.name,
    c.registration_date,
    MIN(o.order_date) AS first_order_date,
    MAX(o.order_date) AS last_order_date,
    DATEDIFF(MAX(o.order_date), MIN(o.order_date)) AS lifespan_days,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(i.net_amount) AS total_revenue,
    ROUND(SUM(i.net_amount) / COUNT(DISTINCT o.order_id), 2) AS avg_purchase_value,
    ROUND(COUNT(DISTINCT o.order_id) / NULLIF(DATEDIFF(MAX(o.order_date), MIN(o.order_date)), 0), 2) AS purchase_frequency,
    ROUND(
        (SUM(i.net_amount) / COUNT(DISTINCT o.order_id)) *
        (COUNT(DISTINCT o.order_id) / NULLIF(DATEDIFF(MAX(o.order_date), MIN(o.order_date)), 0)) *
        DATEDIFF(MAX(o.order_date), MIN(o.order_date)), 2
    ) AS clv
FROM cpg_industry.silver.cpg_consumer c
JOIN cpg_industry.silver.cpg_consumer_order o ON c.consumer_id = o.consumer_id
JOIN cpg_industry.silver.cpg_consumer_invoice i ON o.order_id = i.order_id
WHERE c.is_active = TRUE
GROUP BY c.name, c.registration_date;


In [0]:
%sql
-- B2C Out-of-Stock Rate
-- CREATE OR REPLACE TABLE cpg_industry.gold_sales.gold_out_of_stock_rate_b2c AS
SELECT
    CURRENT_DATE() AS report_date,
    COUNT(DISTINCT CASE 
                      WHEN inv.quantity_on_hand = 0 
                           OR LOWER(inv.inventory_status) = 'low stock' 
                      THEN inv.product_id 
                   END) AS stockout_count,
    COUNT(DISTINCT inv.product_id) AS total_skus,
    ROUND(
        (COUNT(DISTINCT CASE 
                           WHEN inv.quantity_on_hand = 0 
                                OR LOWER(inv.inventory_status) = 'low stock' 
                           THEN inv.product_id 
                        END) * 100.0) 
        / COUNT(DISTINCT inv.product_id), 2
    ) AS out_of_stock_rate
FROM cpg_industry.silver.cpg_inventory AS inv
JOIN cpg_industry.silver.cpg_product AS p 
    ON inv.product_id = p.product_id
WHERE LOWER(p.product_status) = 'active';


### Sales by category

In [0]:
%sql
-- CREATE OR REPLACE TABLE cpg_industry.gold_sales.sales_by_category AS
SELECT 
    p.category AS product_category,
    ROUND(SUM(oi.total_price), 2) AS total_revenue
FROM cpg_industry.silver.cpg_consumer_order_items oi
JOIN cpg_industry.silver.cpg_product p
    ON oi.product_id = p.product_id
JOIN cpg_industry.silver.cpg_consumer_order o
    ON oi.order_id = o.order_id
JOIN  cpg_industry.silver.cpg_consumer_invoice i
    ON o.order_id = i.order_id
WHERE o.order_status = 'Delivered'
GROUP BY p.category
ORDER BY total_revenue DESC;